# Trabalho prático da disciplina Mineração de Dados

## Alunos:
- ### Pedro Augusto Sousa Gonçalves, 21.1.4015
- ### Tiago Gomes da Silva - 19.2.4009

#### Competição: https://www.kaggle.com/competitions/march-machine-learning-mania-2025
#### Repositorio: https://github.com/pedroasgDEV/NCAA_DataMine

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk